In [13]:
import requests
from mastodon import Mastodon, StreamListener
import html2text
import pytz
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from scipy.special import expit

# Mastodon Access Token and Base URL
MASTODON_ACCESS_TOKEN = "Q8oYzpl2vDz6-SOcyf2upfwpUiKS-K9N4qTVLyo2qfA"
MASTODON_BASE_URL = 'https://mastodon.au'

# Initialize Mastodon API
m = Mastodon(api_base_url=MASTODON_BASE_URL, access_token=MASTODON_ACCESS_TOKEN)

# Sentiment Analysis and Text-to-Plain Converter
nltk.download('vader_lexicon', quiet=True)
sid = SentimentIntensityAnalyzer()
html_converter = html2text.HTML2Text()
html_converter.ignore_links = True

# Load Topic Classification Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
class_mapping = model.config.id2label


# Function for Processing Toots
def toot_processing(raw_toot):
    # Convert and Reformat DateTime to Australia/Sydney Timezone
    sydney_timezone = pytz.timezone('Australia/Sydney')
    sydney_datetime = raw_toot['created_at'].astimezone(sydney_timezone)
    created_at = sydney_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

    # Extract Text Content and Analyze Sentiment
    content = html_converter.handle(raw_toot['content']).replace("\n", "")
    sentiment_score = sid.polarity_scores(content)["compound"]

    tokens = tokenizer(content, return_tensors='pt', max_length=512, truncation=True).to(device)
    output = model(**tokens)
    scores = expit(output["logits"][0].detach().cpu().numpy())
    topics = [class_mapping[i] for i, prediction in enumerate((scores >= 0.5) * 1) if prediction]

    return {
        'id': raw_toot['id'],
        'created_at': created_at,
        'content': content,
        'sentiment_score': sentiment_score,
        "topics": topics,
        'url': raw_toot['url'],
        'language': raw_toot['language']
    }

# Listener Class for Streaming Toots
class Listener(StreamListener):
    def __init__(self, upper_limit):
        super().__init__()
        self.count = 0
        self.upper_limit = upper_limit

    def on_update(self, status):
        if status["language"] == "en":
            toot_processed = toot_processing(status)
            print(toot_processed)
            self.count += 1
            if self.count % 50 == 0:
                print(f"Has harvested {self.count} toots.")
            if self.count >= self.upper_limit:
                raise StopStreamingException("Stopping streaming.")


# Custom Exception to Stop Streaming
class StopStreamingException(Exception):
    pass
    
# Main Execution
if __name__ == '__main__':
    header = {"Authorization": f"Bearer {MASTODON_ACCESS_TOKEN}"}
    r = requests.get(f"{MASTODON_BASE_URL}/api/v1/accounts/verify_credentials", headers=header)
    print(r.json())
    listener = Listener(10000)
    try:
        m.stream_public(listener)
    except StopStreamingException as e:
        print(e)




{'id': '112343906767590652', 'username': 'zheyuanw', 'acct': 'zheyuanw', 'display_name': '', 'locked': False, 'bot': False, 'discoverable': None, 'group': False, 'created_at': '2024-04-27T00:00:00.000Z', 'note': '', 'url': 'https://mastodon.au/@zheyuanw', 'uri': 'https://mastodon.au/users/zheyuanw', 'avatar': 'https://mastodon.au/avatars/original/missing.png', 'avatar_static': 'https://mastodon.au/avatars/original/missing.png', 'header': 'https://mastodon.au/headers/original/missing.png', 'header_static': 'https://mastodon.au/headers/original/missing.png', 'followers_count': 0, 'following_count': 0, 'statuses_count': 0, 'last_status_at': None, 'noindex': True, 'source': {'privacy': 'public', 'sensitive': False, 'language': None, 'note': '', 'fields': [], 'follow_requests_count': 0, 'hide_collections': None, 'discoverable': None, 'indexable': False}, 'emojis': [], 'roles': [], 'fields': [], 'role': {'id': '-99', 'name': '', 'permissions': '0', 'color': '', 'highlighted': False}}
{'id': 

KeyboardInterrupt: 

In [8]:
import requests,json
url="https://gateway.api.epa.vic.gov.au/environmentMonitoring/v1/sites"
params={"environmentalSegment":"air"}
headers={
    'User-agent':'curl/8.4.0',
    'Cache-Control':'no-cache',
    'X-API-Key':'f6694fb4cb45496a816c8b630e885f92',
}
response=requests.get(url,params=params,headers=headers)
data=json.loads(response.text)

for record in sorted(data['records'], key=lambda x: x['siteName']):
    print(record['siteName'])
    print(record['geometry']['coordinates'])
    

Alphington
[-37.7784081, 145.0306]
Altona North
[-37.8441238, 144.861618]
Ararat
[-37.2828064, 142.935608]
Bacchus Marsh
[-37.674633, 144.4354]
Bairnsdale
[-37.829895, 147.620209]
Ballarat
[-37.5293274, 143.842438]
Beechworth
[-36.37299, 146.679077]
Benalla
[-36.54932, 145.997223]
Bendigo
[-36.77841, 144.300064]
Boolarra
[-38.36981, 146.2751]
Boolarra South
[-38.44398, 146.2742]
Box Hill
[-37.8287277, 145.1324]
Bright
[-36.73228, 146.9702]
Brighton
[-37.9135475, 144.998]
Broadford
[-37.2078857, 145.04805]
Brooklyn
[-37.8220978, 144.8471]
Callignee
[-38.36169, 146.588211]
Camperdown
[-38.2248344, 143.140152]
Castlemaine
[-37.0730247, 144.233292]
Churchill
[-38.3043137, 146.414932]
Cobden
[-38.32716, 143.076]
Colac
[-38.3456841, 143.594269]
Dandenong
[-37.98576, 145.1987]
Daylesford
[-37.3410339, 144.130463]
Drysdale
[-38.1913528, 144.550659]
Echuca
[-36.12591, 144.754288]
Flynn
[-38.17107, 146.6933]
Flynns Creek
[-38.24396, 146.6362]
Footscray
[-37.80266, 144.8778]
Geelong South
[-38.17

In [39]:
pip install ijson